In [ ]:
from chromadb.utils.embedding_functions import EmbeddingFunction,OpenAIEmbeddingFunction
import chromadb
import os
import openai
from pydantic import BaseModel

CHROMA_PERSIST_DIR = "chroma"

openai.api_key = os.getenv("OPENAI_API_KEY")

# OpenAI client wrapper for Chroma
openai_ef = OpenAIEmbeddingFunction(
                api_key=os.getenv("OPENAI_API_KEY"),
                model_name="text-embedding-3-small"
            )

# Pydantic model for a fact
class Fact(BaseModel):
    id: str
    category: str
    content: str
    score: float = 0.0  # optional, will be used during query result mapping

# Initialize Chroma client and collection
client = chromadb.PersistentClient(path=CHROMA_PERSIST_DIR)
# client = chromadb.EphemeralClient()
collection = client.get_or_create_collection(
    name="facts_collection",
    embedding_function=openai_ef,
    # configuration={"hnsw":{"space":"ip"}}
)

def add_facts_to_collection(facts: list[Fact]):
    documents = [fact.content for fact in facts]
    ids = [str(fact.id) for fact in facts]
    metadatas = [{"category": fact.category} for fact in facts]
    collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [ ]:
[
  {
    "chunk": "余市蒸溜所は、1934年に竹鶴政孝によって設立されました。スコットランドの伝統的な製法を忠実に守り、石炭直火蒸溜という極めて珍しい方法を今なお採用しています。余市の冷涼な気候、澄んだ空気、良質な水、そして海に近い立地が、力強く重厚なピート香とフルーティーな香りを持つウイスキーを生み出しています。"
  },
  {
    "chunk": "余市のシングルモルトは、スモーキーでオイリー、かつしっかりとしたボディが特徴です。特に石炭直火蒸溜による香ばしさと、バーボン樽やシェリー樽での熟成による複雑な風味が、世界中のウイスキー愛好家から高く評価されています。\n\n代表的なテイスティングノート：\n- 香り：ピート、スモーク、青リンゴ、柑橘、バニラ\n- 味わい：トースト、ナッツ、ドライフルーツ、潮風\n- 余韻：長く、スパイシーで温かみがある"
  },
  {
    "chunk": "The Yoichi Distillery, founded in 1934 by Masataka Taketsuru, is renowned for its adherence to traditional Scottish methods, most notably coal-fired direct distillation—a technique rarely seen today. The distillery’s coastal location, pure water sources, and cool climate contribute to the robust, peaty, and complex character of Yoichi single malts."
  },
  {
    "chunk": "Yoichi whiskies are celebrated for their bold smokiness, oily texture, and full-bodied profile. The use of ex-bourbon and sherry casks for maturation adds layers of vanilla, dried fruit, and subtle maritime notes. The distillery’s commitment to craftsmanship and authenticity has earned it numerous international awards, including World’s Best Single Malt at the World Whiskies Awards.\n\nTasting Notes:\n- Nose: Peat smoke, brine, orchard fruits, citrus, vanilla\n- Palate: Toasted nuts, dried fruits, spice, sea breeze\n- Finish: Long, warming, with lingering spice and smoke"
  },
  {
    "chunk": "蒸馏所研学预定：https://distillery.nikka.com/yoichi/reservation \n 专业说明\n- 余市蒸溜所目前仍然是日本极少数采用石炭直火蒸溜的酒厂，这种工艺能够赋予酒体独特的厚重感和烟熏风味。\n- 代表作如Single Malt Yoichi、Yoichi 10年、Yoichi Peated等，均在国际大赛中屡获殊荣。\n- 参观余市蒸溜所时，专业客人可特别关注其发酵罐（木桶发酵）、蒸馏器形状、以及不同熟成仓库的温湿度管理。"
  }
]


In [6]:
rules = [
  {
    "chunk": "Yoichi Nap is a villa hotel located in 5 Chome-32 Okawacho, Yoichi, Yoichi District, Hokkaido 046-0004, Japan. [Google Map location](https://maps.app.goo.gl/7HyR1H2obCsQTzZj8) "
  },
  {"chunk": "The villa with 3 bedrooms, 2 living rooms, and 3 bathrooms."},
  {
    "chunk": "It comfortably accommodates 7 people, with a maximum occupancy of 10."
  },
  {
    "chunk": "There are 2 adult bedrooms, each with a king-size bed, and 1 bedroom with 3 full-size beds—ideal for kids but suitable for adults too."
  },
  {
    "chunk": "Amenities include a kitchen, minibar, TV, air conditioning, heating, garden, private parking, EV charging, WiFi-7, hot tub, washing machine, and cribs."
  },
  {
    "chunk": "The villa is smoke-free, child-friendly, and includes a bar."
  },
  {
    "chunk": "Check-in starts at 16:00 and check-out is by 11:00. later check-out may not available now. early check-in may available."
  },
  {
    "chunk": "Breakfast is not provided, but full kitchenware is available for homemade meals using ingredients from local markets."
  },
  {
    "chunk": "Pet policy : Pets are not allowed."
  },
  {
    "chunk": "The property is not wheelchair accessible and has no elevator."
  },
  {
    "chunk": "Free private parking is available along with EV charging."
  },
  {
    "chunk": "Guests enjoy free high-speed WiFi-7 internet."
  },
  {
    "chunk": "Nearby attractions include Hamanaka Moire Beach (19-minute walk), Otaru Station (12 miles), and Otaru Canal Park (12 miles)."
  },
  {
    "chunk": "Other notable locations nearby are Otarushi Zenibako City Center (22 miles) and Okadama Airport (35 miles)."
  },
  {
    "chunk": "The license number for the property is 後保生第187号令."
  },
  {
    "chunk": "The minimum stay is 1 night."
  },
  {
    "chunk": "Cancellation is free, but guests should confirm details before booking."
  },
  {
    "chunk": "Yoichi Nap is family-friendly and provides accommodations and amenities suitable for children."
  },{"chunk": "Hotel Website: https://yoichi-nap.com/"}
]


In [7]:
from typing import List
import uuid
x:List[Fact] = []
for r in rules:
    x.append(Fact(id=str(uuid.uuid4()),category="Rule",content=r["chunk"]))
add_facts_to_collection(x)

[2025-06-22 18:23:44 - openai._base_client:482 - DEBUG] Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'idempotency_key': 'stainless-python-retry-a46e81fa-984c-4e8f-976f-6c5dd826269d', 'post_parser': <function Embeddings.create.<locals>.parser at 0x7f3f7c3ce7a0>, 'json_data': {'input': ['Yoichi Nap is a villa hotel located in 5 Chome-32 Okawacho, Yoichi, Yoichi District, Hokkaido 046-0004, Japan. [Google Map location](https://maps.app.goo.gl/7HyR1H2obCsQTzZj8) ', 'The villa with 3 bedrooms, 2 living rooms, and 3 bathrooms.', 'It comfortably accommodates 7 people, with a maximum occupancy of 10.', 'There are 2 adult bedrooms, each with a king-size bed, and 1 bedroom with 3 full-size beds—ideal for kids but suitable for adults too.', 'Amenities include a kitchen, minibar, TV, air conditioning, heating, garden, private parking, EV charging, WiFi-7, hot tub, washing machine, and cribs.', 'The villa is smoke-free, child-friendly, and includes a bar.', 'Check-in sta

In [ ]:
[
  {
    "chunk": "新千岁机场到余市主要有三种方式：电车（JR）、巴士和自驾。以下为详细说明：\n\n1. JR电车路线（推荐，方便快捷）\n在新千岁机场地下1楼搭乘JR千岁线（快速Airport号）前往札幌站，约40分钟。\n札幌站换乘JR函馆本线，前往小樽站，约30分钟。\n小樽站再换乘JR函馆本线（往俱知安方向），约25分钟到达余市站。\n全程约2小时30分钟，票价约2,440日元。\n示例时刻表：07:33 新千岁机场出发 → 10:01 余市站到达（约2小时28分钟，2次换乘）。"
  },
  {
    "chunk": "2. 巴士路线\n从新千岁机场搭乘机场巴士到札幌市区（大通公园），约1小时6分钟。\n在札幌市区（时计台前）换乘开往岩内的中央巴士，在余市站前十字街下车，约1小时36分钟。\n总时长约2小时40分钟，适合携带大件行李或希望直达市区的旅客。"
  },
  {
    "chunk": "3. 自驾路线\n从新千岁机场租车，走道央高速和札樽高速，约120公里，车程约2小时30分钟。\n适合家庭或多人同行，沿途风景优美，灵活度高。"
  },
  {
    "chunk": "小贴士：\n建议优先选择JR电车，班次多，准点，换乘方便。\n若携带大件行李或希望直达，可选择巴士。\n自驾适合自由安排行程的旅客。"
  }
]


In [ ]:
from typing import List
import mcpx
from mcpx import Fact
import uuid

# Define the Q&A content first
qa_contents = [
    {"chunk": "Q: 如何预订房间？ A: 您可以通过多种方式预订：官网预订(yoichi-nap.com)、OTA平台（AirBnb, Booking.com, trip.com）、社交媒体平台或直接到店预订。"},
    {"chunk": "Q: 预订后可以更改或取消吗？ A: 可以更改或取消预订。如果是直接电话预订，可直接联系酒店；如果通过网上或旅行社预订，请在相应平台操作或联系旅行社。"},
    {"chunk": "Q: 取消政策是什么？ A: 一般取消政策为：入住前一日取消收取20%取消费，入住当天取消收取80%取消费，无故未到收取100%取消费。"},
    {"chunk": "Q: 入住和退房时间是什么时候？ A: 入住时间通常从下午3点开始，退房时间截至上午11点。"},
    {"chunk": "Q: 可以提前入住或延迟退房吗？ A: 可以申请提前入住或延迟退房，但需视房间供应情况而定，可能产生额外费用。"},
    {"chunk": "Q: 入住时需要什么证件？ A: 通过AI办理checkin，拍照上传有效证件即可，或者通过H5页面提交"},
    {"chunk": "Q: 房间内有保险箱吗？ A: 是的，所有房间都配有保险箱。"},
    {"chunk": "Q: 房间有空调/暖气吗？ A: 有，住客可以按需开放"},
    {"chunk": "Q: 房间内可以上网吗？ A: Softbank 千兆光纤， wifi7 网络覆盖，"},
    {"chunk": "Q: 可以要求特定房间号码吗？ A: 该物件仅有一间，并无房间号"},
    {"chunk": "Q: 可以带客人进入房间吗？ A: 可以，但不得超过房间接待能力。额外人员需到需要在checkin时登记"},
    {"chunk": "Q: 可以加床吗？ A: 不支持加床服务，三间卧室总共已经提供5张床"},
    {"chunk": "Q: 房间打扫服务如何安排？ A: 会在不打扰客人的情况下进行清洁，如客人在场需征得同意。对私人用品会小心整理，必要时通过纸条沟通。"},
    {"chunk": "Q: 预订后可以添加早餐吗？ A: 酒店提供餐具和厨具"},
    {"chunk": "Q: 错过早餐时间怎么办？ A: 提供包含丰富零食和饮料的minibar"},
    {"chunk": "Q: 儿童入住政策是什么？ A: 5岁以下儿童与成人同睡不收费，6岁以上儿童房费与成人相同。提供儿童专属服务如宝宝餐具、儿童椅等。"},
    {"chunk": "Q: 可以携带宠物吗？ A: 一般不接受宠物入住，但导盲犬、导聋犬等持照协助犬除外。"},
    {"chunk": "Q: 未成年人可以单独入住吗？ A: 未满18岁未成年人单独入住需要法定监护人签署的同意书。"},
    {"chunk": "Q: 遇到紧急情况怎么办？ A: 酒店提供24小时前台服务，遇到生病、设备故障等紧急情况可随时联系前台处理。"},
    {"chunk": "Q: 对服务不满意如何投诉？ A: 酒店建立完善的客户投诉反馈机制，会以住客立场积极处理问题，并提供事后补偿措施。"}
]

# Create Facts list and populate it
x: List[Fact] = []
for qa in qa_contents:
    x.append(Fact(id=str(uuid.uuid4()), category="FAQ", content=qa["chunk"]))

# Add facts to the collection
mcpx.add_facts_to_collection(x)


In [1]:
images_result = """{
    "translate": true,
    "success": true,
    "message": "操作成功！",
    "code": 200,
    "rtnCode": "200",
    "result": [
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577640841217",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1748164897960.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577661812737",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/a38d461d24e96016feb6b6d14db03fcc_1748164960302.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577682784257",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/f0f421418433ba3cb592238eb7e51441_1748164980949.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577707950081",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ab35e84a215f0f711ed629c2abb9efa0_1748164987505.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577728921602",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/c4ca4238a0b923820dcc509a6f75849b_1748165003477.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577745698817",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/34b8a5e9772cd73daba26c68a209127f_1748165053306.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577766670337",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/99cdc1a6790c69c2a6f113245c0df55b_1748165141427.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577787641857",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/611814ce577dee3a875e04ddf24a4ed3_1748165153324.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577804419074",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/c1a630111e7d771acef16ac08e05e6d5_1748165173677.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:26:34",
            "fileType": "1",
            "id": "1926570577825390593",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/c81e728d9d4c2f636f067f89cc14862c_1748165191526.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:58:17",
            "fileType": "1",
            "id": "1926578557207339010",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/2cb17e69baebaeed67190ad729bb2601_1748167008196.PNG",
            "pickUp": false,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:58:17",
            "fileType": "1",
            "id": "1926578557219921922",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1748167018862.PNG",
            "pickUp": false,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:58:17",
            "fileType": "1",
            "id": "1926578557228310529",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/f0f421418433ba3cb592238eb7e51441_1748167030262.PNG",
            "pickUp": false,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:58:17",
            "fileType": "1",
            "id": "1926578557240893442",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1748167052652.PNG",
            "pickUp": false,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 18:58:57",
            "fileType": "1",
            "id": "1926578726627860481",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1748167135248.PNG",
            "pickUp": false,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-05-25 19:04:35",
            "fileType": "1",
            "id": "1926580141694742529",
            "isVertical": 0,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/5d58935edc001fa58665a1e463cba6d6_1748167471769.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987678683137",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/c4ca4238a0b923820dcc509a6f75849b_1749033252733.png",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987691266050",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ab35e84a215f0f711ed629c2abb9efa0_1749033284929.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987703848961",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/77c77478b552f1977cad320741ba92a5_1749033304375.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987720626177",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ab35e84a215f0f711ed629c2abb9efa0_1749033313483.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987729014786",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/f0f421418433ba3cb592238eb7e51441_1749033318504.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987749986306",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1749033344912.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987758374914",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/daa8f13c9d65da091604a1c944e47f3b_1749033351230.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        },
        {
            "createBy": "yoichi.nap@gmail.com",
            "createTime": "2025-06-04 19:36:14",
            "fileType": "1",
            "id": "1930211987770957825",
            "isVertical": 1,
            "name": null,
            "path": "https://minio.onesteppms.com/bk1/temp/c4ca4238a0b923820dcc509a6f75849b_1749033360945.PNG",
            "pickUp": true,
            "sysOrgCode": null,
            "tag": null,
            "typeId": "1913452442486317057",
            "updateBy": null,
            "updateTime": null,
            "url": null,
            "videoImg": ""
        }
    ],
    "timestamp": 1749539829922
}
"""
import json
image_urls = []
for item in json.loads(images_result).get("result", []):
    if item.get("fileType") == "1" and "path" in item:
        url = item["path"]
        image_urls.append(url)

In [2]:
image_urls

['https://minio.onesteppms.com/bk1/temp/ec308451c1d095c528cfa3c009ea7235_1748164897960.PNG',
 'https://minio.onesteppms.com/bk1/temp/a38d461d24e96016feb6b6d14db03fcc_1748164960302.PNG',
 'https://minio.onesteppms.com/bk1/temp/f0f421418433ba3cb592238eb7e51441_1748164980949.PNG',
 'https://minio.onesteppms.com/bk1/temp/ab35e84a215f0f711ed629c2abb9efa0_1748164987505.PNG',
 'https://minio.onesteppms.com/bk1/temp/c4ca4238a0b923820dcc509a6f75849b_1748165003477.PNG',
 'https://minio.onesteppms.com/bk1/temp/34b8a5e9772cd73daba26c68a209127f_1748165053306.PNG',
 'https://minio.onesteppms.com/bk1/temp/99cdc1a6790c69c2a6f113245c0df55b_1748165141427.png',
 'https://minio.onesteppms.com/bk1/temp/611814ce577dee3a875e04ddf24a4ed3_1748165153324.png',
 'https://minio.onesteppms.com/bk1/temp/c1a630111e7d771acef16ac08e05e6d5_1748165173677.png',
 'https://minio.onesteppms.com/bk1/temp/c81e728d9d4c2f636f067f89cc14862c_1748165191526.png',
 'https://minio.onesteppms.com/bk1/temp/2cb17e69baebaeed67190ad729bb26